In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<h1> Import Libraries Here </h1>


In [ ]:
import pandas as pd
import numpy as np
import json
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
from gensim.models import Word2Vec
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence


<h1> Read Training Data Here </h1>

In [ ]:
prepared_dataset=pd.read_csv('/content/drive/MyDrive/Math23K_English_with_data_prep.csv')
#prepared_dataset.head()

In [ ]:
with open('/content/drive/MyDrive/equations_data.json') as f:
  prefix_equations_list = json.load(f)
#print(len(prefix_equations_list))
#print(prefix_equations_list[0])

<h1> Extract Parts Of Prepared Dataset Here

In [ ]:
prepared_questions=prepared_dataset['prepared_question'].values.tolist()
#print(len(prepared_questions))
#print(prepared_questions[0])   

In [ ]:
normal_questions=prepared_dataset['segmented_question'].values.tolist()
#print(len(normal_questions))
#print(normal_questions[0])

In [ ]:
infix_equations=prepared_dataset['equation'].values.tolist()
#print(len(infix_equations))
#print(infix_equations[0])

In [ ]:
answers=prepared_dataset['ans'].values.tolist()
#print(len(answers))
#print(answers[0])

<h1> Preprocess The Data

<h2> Remove the unnecessary Data (No Prefix) </h2> 

In [ ]:
idx_to_remove=[i for i in range(len(prefix_equations_list)) if prefix_equations_list[i]=="no_prefix"]
prepared_questions=[prepared_questions[i] for i in range(len(prefix_equations_list)) if prefix_equations_list[i] != "no_prefix"]
normal_questions=[normal_questions[i] for i in range(len(prefix_equations_list)) if prefix_equations_list[i] != "no_prefix"]
infix_equations=[infix_equations[i] for i in range(len(prefix_equations_list)) if prefix_equations_list[i] != "no_prefix"]
answers=[answers[i] for i in range(len(prefix_equations_list)) if prefix_equations_list[i] != "no_prefix"]
prefix_equations_list=[equation for equation in prefix_equations_list if equation != "no_prefix"]


idx_to_remove_chinnese=[i for i in range(len(infix_equations)) if infix_equations[i]=="x=80千米/小时"]
print(idx_to_remove_chinnese)
if len(idx_to_remove_chinnese) >=1:
  prepared_questions.pop(idx_to_remove_chinnese[0])
  normal_questions.pop(idx_to_remove_chinnese[0])
  infix_equations.pop(idx_to_remove_chinnese[0])
  answers.pop(idx_to_remove_chinnese[0])
  prefix_equations_list.pop(idx_to_remove_chinnese[0])

# print(len(idx_to_remove))
# print(idx_to_remove)
# print(len(prefix_equations_list))
# print(len(prepared_questions))
# print(len(normal_questions)) 
# print(len(infix_equations))
# print(len(answers))

[10149]


In [ ]:
# print(prefix_equations_list[2578])
# print(prepared_questions[2578])
# print(normal_questions[2578])
# print(infix_equations[2578])
# print(answers[2578])

<h2> Tokenize  </h2>

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
temp_questions=list()
for elem in prepared_questions:
  temp_questions.append(word_tokenize(elem))
prepared_questions=temp_questions
#print(len(prepared_questions))
#print(prepared_questions[0])


<h1> Create Vocab Input and Output Both</h1>

In [ ]:
index_to_word_input=['<pad>']
for question in prepared_questions:
  for word in question:
    if(word not in index_to_word_input):
      index_to_word_input.append(word)
#print(len(index_to_word_input))
#print(index_to_word_input[0])
input_vocab_size=len(index_to_word_input)
input_vocab_size

7805

In [ ]:
output_vocab_size

33

In [ ]:
word_to_index_input={'<pad>':0}
for question in prepared_questions:
  for word in question:
    if(word not in word_to_index_input.keys()):
      idx=index_to_word_input.index(word)
      word_to_index_input[word]=idx
#print(len(word_to_index_input))
#print(word_to_index_input["n0"])
#print(word_to_index_input["n1"])
#print(word_to_index_input["children"])
#print(word_to_index_input['<pad>'])


In [ ]:
index_to_word_output=['<pad>','<try>']
for equation in prefix_equations_list:
  for elem in equation:
    if elem not in index_to_word_output:
      index_to_word_output.append(elem)
# print(len(index_to_word_output))
# print(index_to_word_output[0])
index_to_word_output.append('<start>')
index_to_word_output.append('<end>')
output_vocab_size=len(index_to_word_output)
print(output_vocab_size)
print(index_to_word_output)

33
['<pad>', '<try>', '*', '-', 'n1', '1', 'n0', '+', 'n2', '/', 'n4', 'n3', 'n6', '7', 'n5', 'n7', '3', '6', '5', '0', '4', '2', '8', '^', '9', 'n8', 'n9', 'n10', 'n11', 'n13', 'n15', '<start>', '<end>']


In [ ]:
word_to_index_output={'<pad>':0,'<try>':1}
for word in index_to_word_output:
  if word not in word_to_index_output.keys():
    word_to_index_output[word]=index_to_word_output.index(word)
print(len(word_to_index_output))
print(word_to_index_output)

33
{'<pad>': 0, '<try>': 1, '*': 2, '-': 3, 'n1': 4, '1': 5, 'n0': 6, '+': 7, 'n2': 8, '/': 9, 'n4': 10, 'n3': 11, 'n6': 12, '7': 13, 'n5': 14, 'n7': 15, '3': 16, '6': 17, '5': 18, '0': 19, '4': 20, '2': 21, '8': 22, '^': 23, '9': 24, 'n8': 25, 'n9': 26, 'n10': 27, 'n11': 28, 'n13': 29, 'n15': 30, '<start>': 31, '<end>': 32}


<h1> Encode Data </h1>

In [ ]:
prepared_questions_temp=list()
for question in prepared_questions:
  question_temp=list()
  for word in question:
    question_temp.append(word_to_index_input[word])
  prepared_questions_temp.append(question_temp)
# print(prepared_questions[11235])
prepared_questions=prepared_questions_temp
# print(prepared_questions[11235])

<h1> Padding </h1>

In [ ]:
max_len=max(len(question) for question in prepared_questions)
#print(max_len)
prepared_questions= sequence.pad_sequences(prepared_questions, maxlen=max_len,padding="post")
# print(prepared_questions[2])
# print(len(prepared_questions))
#reviews.shape

<h1> Encode Equations </h1>

In [ ]:
prefix_equations_list_temp=list()
for equation in prefix_equations_list:
  equation_temp=[word_to_index_output['<start>']]
  for elem in equation:
    equation_temp.append(word_to_index_output[elem])
  equation_temp.append(word_to_index_output['<end>'])
  prefix_equations_list_temp.append(equation_temp)
prefix_equations_list=prefix_equations_list_temp
#print(prefix_equations_list[0])
#print(prefix_equations_list[2])

<h1> Padding Equations</h1>

In [ ]:
max_len=max(len(equation) for equation in prefix_equations_list)
#print(max_len)
prefix_equations_list= sequence.pad_sequences(prefix_equations_list, maxlen=max_len,padding="post")
#print(prefix_equations_list[0])
#print(prefix_equations_list[2])
#print(len(prefix_equations_list))
#reviews.shape

<h1> Import Pytorch/Tensorflow Dependencies Here

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader 
import torch.nn.functional as F
from torch.utils.data.sampler import WeightedRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
import collections
import logging
import os
import pathlib
import re
import string
import sys
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
#import tensorflow_text as text
import tensorflow as tf

<h1> Split Data into Train and Validation and Test</h1>

In [ ]:
prepared_questions_train, prepared_questions_test, prefix_equations_list_train, prefix_equations_list_test = train_test_split(prepared_questions,prefix_equations_list, train_size=0.8)


In [ ]:
#prepared_questions_train, prepared_questions_test, prefix_equations_list_train, prefix_equations_list_test = train_test_split(prepared_questions,prefix_equations_list, train_size=0.9)
## print(len(prepared_questions_train))
## print(len(prefix_equations_list_train))
## print(len(prepared_questions_test))
## print(len(prefix_equations_list_test))

In [ ]:
#prepared_questions_train, prepared_questions_val, prefix_equations_list_train, prefix_equations_list_val = train_test_split(prepared_questions_train,prefix_equations_list_train, train_size=0.9)

In [ ]:
BUFFER_SIZE = len(prepared_questions_train)
BATCH_SIZE = 64
steps_per_epoch = len(prepared_questions_train)//BATCH_SIZE
dataset = tf.data.Dataset.from_tensor_slices((prepared_questions_train, prefix_equations_list_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
input_vocab_size = input_vocab_size
target_vocab_size = 42
dropout_rate = 0.0

In [ ]:
# We provide positional information about the data to the model,
# otherwise each sentence will be treated as Bag of Words
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

In [ ]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
  
  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
  
  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    
  pos_encoding = angle_rads[np.newaxis, ...]
    
  return tf.cast(pos_encoding, dtype=tf.float32)

#### Masking

In [ ]:
# mask all elements are that not words (padding) so that it is not treated as input
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [ ]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask

In [ ]:
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)


#### Attention

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  
  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)
        
  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
    
  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]
    
    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)
    
    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
    
    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)
    
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
    return output, attention_weights

#### Pointwise Feed forward network

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

#### Encoder Layer

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    # normalize data per feature instead of batch
    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    
  def call(self, x, training, mask):
    # Multi-head attention layer
    attn_output, _ = self.mha(x, x, x, mask) 
    attn_output = self.dropout1(attn_output, training=training)
    # add residual connection to avoid vanishing gradient problem
    out1 = self.layernorm1(x + attn_output)
    
    # Feedforward layer
    ffn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output, training=training)
    # add residual connection to avoid vanishing gradient problem
    out2 = self.layernorm2(out1 + ffn_output)
    return out2

#### Encoder

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                            self.d_model)
    
    # Create encoder layers (count: num_layers)
    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
  
    self.dropout = tf.keras.layers.Dropout(rate)
        
  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]

    # adding embedding and position encoding.
    x = self.embedding(x)  
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)
    
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)
    
    return x 

#### Decoder Layer

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)
 
    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
  def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):

    # Masked multihead attention layer (padding + look-ahead)
    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
    attn1 = self.dropout1(attn1, training=training)
    # again add residual connection
    out1 = self.layernorm1(attn1 + x)
    
    # Masked multihead attention layer (only padding)
    # with input from encoder as Key and Value, and input from previous layer as Query
    attn2, attn_weights_block2 = self.mha2(
        enc_output, enc_output, out1, padding_mask)
    attn2 = self.dropout2(attn2, training=training)
    # again add residual connection
    out2 = self.layernorm2(attn2 + out1)
    
    # Feedforward layer
    ffn_output = self.ffn(out2)
    ffn_output = self.dropout3(ffn_output, training=training)
    # again add residual connection
    out3 = self.layernorm3(ffn_output + out2)
    return out3, attn_weights_block1, attn_weights_block2

#### Decoder

In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
     
    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
    
    # Create decoder layers (count: num_layers)
    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)
    
  def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}
    
    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    
    x += self.pos_encoding[:,:seq_len,:]
    
    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)
      
      # store attenion weights, they can be used to visualize while translating
      attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
      attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
    return x, attention_weights

#### Transformer

In [ ]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, 
               target_vocab_size, pe_input, pe_target, rate=0.1):
    super(Transformer, self).__init__()

    self.encoder = Encoder(num_layers, d_model, num_heads, dff, 
                           input_vocab_size, pe_input, rate)

    self.decoder = Decoder(num_layers, d_model, num_heads, dff, 
                           target_vocab_size, pe_target, rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
  def call(self, inp, tar, training, enc_padding_mask, 
           look_ahead_mask, dec_padding_mask):

    # Pass the input to the encoder
    enc_output = self.encoder(inp, training, enc_padding_mask)
    
    # Pass the encoder output to the decoder
    dec_output, attention_weights = self.decoder(
        tar, enc_output, training, look_ahead_mask, dec_padding_mask)
    
    # Pass the decoder output to the last linear layer
    final_output = self.final_layer(dec_output)
    
    return final_output, attention_weights

#### Optimizer and Loss

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps
    
  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    
    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
learning_rate = CustomSchedule(d_model)

# Adam optimizer with a custom learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [ ]:
def loss_function(real, pred):
  # Apply a mask to paddings (0)
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  
  return tf.reduce_mean(loss_)

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

In [ ]:
transformer = Transformer(num_layers, d_model, num_heads, dff,
                          input_vocab_size, target_vocab_size, 
                          pe_input=input_vocab_size, 
                          pe_target=target_vocab_size,
                          rate=dropout_rate)

In [ ]:
def create_masks(inp, tar):
  # Encoder padding mask
  enc_padding_mask = create_padding_mask(inp)
  
  # Decoder padding mask
  dec_padding_mask = create_padding_mask(inp)
  
  # Look ahead mask (for hiding the rest of the sequence in the 1st decoder attention layer)
  look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
  dec_target_padding_mask = create_padding_mask(tar)
  combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
  return enc_padding_mask, combined_mask, dec_padding_mask

### Checkpoints

In [ ]:
# !rm -r /gdrive/My\ Drive/ADL\ Project/checkpoints/training_checkpoints/akshata_transfomer

In [ ]:
checkpoint_dir = os.path.join("/content/drive/MyDrive/checkpoints")
checkpoint_dir = os.path.join("/content/drive/MyDrive/checkpoints/math_bot")

print("Checkpoints directory is", checkpoint_dir)
if os.path.exists(checkpoint_dir):
  print("Checkpoints folder already exists")
else:
  print("Creating a checkpoints directory")
  os.makedirs(checkpoint_dir)


checkpoint = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=5)

Checkpoints directory is /content/drive/MyDrive/checkpoints/math_bot
Checkpoints folder already exists


In [ ]:
latest = ckpt_manager.latest_checkpoint
latest

'/content/drive/MyDrive/checkpoints/math_bot/ckpt-5'

In [ ]:
if latest:
  epoch_num = int(latest.split('/')[-1].split('-')[-1])
  checkpoint.restore(latest)
  print ('Latest checkpoint restored!!')
else:
  epoch_num = 0

Latest checkpoint restored!!


In [ ]:
epoch_num

5

#### Training

In [ ]:
EPOCHS = 5

def train_step(inp, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]
  
  enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)
  
  with tf.GradientTape() as tape:
    predictions, _ = transformer(inp, tar_inp,  True, enc_padding_mask, combined_mask, dec_padding_mask)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)    
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
  
  train_loss(loss)
  #print(tar_real)
  #print("----------------------")
  #print(predictions)
  train_accuracy(tar_real, predictions)

In [ ]:
for epoch in range(epoch_num, EPOCHS):
  start = time.time()
  
  train_loss.reset_states()
  train_accuracy.reset_states()
  
  # inp -> question, tar -> equation
  for (batch, (inp, tar)) in enumerate(dataset):
    #print(inp.shape)
    #print(tar.shape)
    train_step(inp, tar)
    
    if batch % 50 == 0:
      print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
          epoch + 1, batch, train_loss.result(), train_accuracy.result()))
      
  ckpt_save_path = ckpt_manager.save()
  print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    
  print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                train_loss.result(), 
                                                train_accuracy.result()))

  print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

In [ ]:
#jiang 
inp_sentence_words="troy   had  n0  apple . he gave n1 apple to prot . how many does he have left"
inp_sentence=list()
inp_sentence_words_token=word_tokenize(inp_sentence_words)
for elem in inp_sentence_words_token:
  try:
    inp_sentence.append(word_to_index_input[elem])
  except:
    inp_sentence.append(0)
to_append_length=135-len(inp_sentence)
for i in range(0,to_append_length):
  inp_sentence.append(0)
print(len(inp_sentence))
print(inp_sentence)

135
[0, 114, 20, 746, 16, 64, 428, 33, 746, 9, 0, 16, 34, 35, 80, 64, 81, 148, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
MAX_LENGTH=42
# inp_sentence=[11, 1770,  643, 1771, 1052,  751, 1575,   28,   51,  567,   20,71,  141,   23,   33, 1575,  148,   16,   34,   35, 1575,   80,   37,
#         1770,  643, 1771,   81,   39,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0]
print(len(inp_sentence))
encoder_input = tf.expand_dims(inp_sentence, 0)
decoder_input = [word_to_index_output['<start>']]
output = tf.expand_dims(decoder_input, 0)
for i in range(MAX_LENGTH):
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
        encoder_input, output)
    predictions, attention_weights = transformer(encoder_input, 
                                                    output,
                                                    False,
                                                    enc_padding_mask,
                                                    combined_mask,
                                                    dec_padding_mask)
    predictions = predictions[: ,-1:, :] 
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
    
    # return the result if the predicted_id is equal to the end token
    if predicted_id == 32:
      #print(tf.squeeze(output, axis=0), attention_weights)
      break
    
    # concatentate the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)
print(tf.squeeze(output, axis=0))
        
    #print(predictions)
    # select the last word from the seq_len dimension
#predictions = predictions[: ,-1:, :] 
#predictions

135
tf.Tensor([31  3  6  4], shape=(4,), dtype=int32)


In [ ]:
output=tf.squeeze(output, axis=0)
for elem in output:
  if(elem!=31):
    print(index_to_word_output[elem])
  elif elem ==32:
    break

-
n0
n1


In [ ]:
for elem in inp_sentence:
  print(index_to_word_input[elem],end=' ')

<pad> had n0 apple . he gave n1 apple to <pad> . how many does he have left <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

In [ ]:
def predicted_eqn(inp_sentence_words):  
  MAX_LENGTH=42
  input_sentence_token=word_tokenize(inp_sentence_words)
  input_sentence=[]
  for word in input_sentence_token:
    try:
      input_sentence.append(word_to_index_input[word])
    except:
      input_sentence.append(0)
  to_pad=42-len(input_sentence_token)
  for _ in range(0,to_pad):
    input_sentence.append(0)
  #print(input_sentence)
  encoder_input = tf.expand_dims(input_sentence, 0)
  decoder_input = [word_to_index_output['<start>']]
  output = tf.expand_dims(decoder_input, 0)
  for i in range(MAX_LENGTH):
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)
    predictions, attention_weights = transformer(encoder_input, 
                                                  output,
                                                  False,
                                                  enc_padding_mask,
                                                  combined_mask,
                                                  dec_padding_mask)
    predictions = predictions[: ,-1:, :] 
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
    
    # return the result if the predicted_id is equal to the end token
    if predicted_id == word_to_index_output['<end>']:
      #print(tf.squeeze(output, axis=0), attention_weights)
      break
    
    # concatentate the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)
  output=tf.squeeze(output, axis=0)
  output_ret=list()
  for elem in output:
    output_ret.append(index_to_word_output[elem])
  return output_ret[1:]


In [ ]:
for elem in output:
  print(index_to_word_output[elem])

<start>
-
n0
n1


In [ ]:
with open('/content/drive/MyDrive/equations_data.json') as f:
  prefix_equations_list_eval = json.load(f)
#print(len(prefix_equations_list))
#print(prefix_equations_list[0])
prepared_dataset_eval=pd.read_csv('/content/drive/MyDrive/Math23K_English_with_data_prep.csv')
#prepared_dataset.head()
prepared_questions_eval=prepared_dataset_eval['prepared_question'].values.tolist()
#print(len(prepared_questions))
#print(prepared_questions[0])   
normal_questions_eval=prepared_dataset_eval['segmented_question'].values.tolist()
#print(len(normal_questions))
#print(normal_questions[0])
infix_equations_eval=prepared_dataset_eval['equation'].values.tolist()
#print(len(infix_equations))
#print(infix_equations[0])
answers_eval=prepared_dataset_eval['ans'].values.tolist()
#print(len(answers))
#print(answers[0])

In [ ]:
idx_to_remove=[i for i in range(len(prefix_equations_list_eval)) if prefix_equations_list_eval[i]=="no_prefix"]
prepared_questions_eval=[prepared_questions_eval[i] for i in range(len(prefix_equations_list_eval)) if prefix_equations_list_eval[i] != "no_prefix"]
normal_questions_eval=[normal_questions_eval[i] for i in range(len(prefix_equations_list_eval)) if prefix_equations_list_eval[i] != "no_prefix"]
infix_equations_eval=[infix_equations_eval[i] for i in range(len(prefix_equations_list_eval)) if prefix_equations_list_eval[i] != "no_prefix"]
answers=[answers_eval[i] for i in range(len(prefix_equations_list_eval)) if prefix_equations_list_eval[i] != "no_prefix"]
prefix_equations_list_eval=[equation for equation in prefix_equations_list_eval if equation != "no_prefix"]


idx_to_remove_chinnese=[i for i in range(len(infix_equations_eval)) if infix_equations_eval[i]=="x=80千米/小时"]
print(idx_to_remove_chinnese)
if len(idx_to_remove_chinnese) >=1:
  prepared_questions_eval.pop(idx_to_remove_chinnese[0])
  normal_questions_eval.pop(idx_to_remove_chinnese[0])
  infix_equations_eval.pop(idx_to_remove_chinnese[0])
  answers_eval.pop(idx_to_remove_chinnese[0])
  prefix_equations_list_eval.pop(idx_to_remove_chinnese[0])

# print(len(idx_to_remove))
# print(idx_to_remove)
# print(len(prefix_equations_list))
# print(len(prepared_questions))
# print(len(normal_questions)) 
# print(len(infix_equations))
# print(len(answers))

[10149]


In [ ]:
def is_operand(c):
    """
    Return True if the given char c is an operand, e.g. it is a number
    """
    reject_list=['*','+','-','/','^']
    if c in reject_list:
      return False
    return True
 
#print(is_operand('1/3'))
def evaluate_prefix(expression):
    """
    Evaluate a given expression in prefix notation.
    Asserts that the given expression is valid.
    """
    stack = []
    #print(expression)
    #print(expression[::-1])
    # iterate over the string in reverse order
    for c in expression[::-1]:
 
        # push operand to stack
        if is_operand(c):
            try:
              stack.append(float(c))
            except:
              stack.append(float(c[0])/float(c[2]))
 
        else:
            # pop values from stack can calculate the result
            # push the result onto the stack again
            o1 = stack.pop()
            o2 = stack.pop()
 
            if c == '+':
                stack.append(o1 + o2)
 
            elif c == '-':
                stack.append(o1 - o2)
 
            elif c == '*':
                stack.append(o1 * o2)
 
            elif c == '/':
                stack.append(o1 / o2)
            elif c == '^':
                stack.append(o1 ** o2)

 
    return stack.pop()
 


In [ ]:
print(input_vocab_size)

7805


In [ ]:




def text2int(textnum, numwords={}):
    flag=False
    if not numwords:
      units = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen"
      ]
      

      tens = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]

      scales = []

      #numwords["and"] = (1, 0)
      for idx, word in enumerate(units):    numwords[word] = (1, idx)
      #for idx, word in enumerate(units1):    numwords[word] = (1, idx)
      for idx, word in enumerate(tens):     numwords[word] = (1, idx * 10)
      for idx, word in enumerate(scales):   numwords[word] = (10 ** (idx * 3 or 2), 0)

    current = result = 0
    for word in textnum.split():
        if word not in numwords:
          flag=True
          return word,flag
        else:
            scale, increment = numwords[word]
            current = current * scale + increment
            if scale > 100:
                result += current
                current = 0

    return (result + current,flag)


def convert_to_lower(sentence):
    return sentence.lower()

def pre_process_sentence(sentence):
    sentence=convert_to_lower(sentence)
    sentence=sentence.strip()
    sentence=sentence.replace("twice","2 times")
    sentence=sentence.replace("thrice","3 times")
    sentence1=""
    
    for word in sentence.split(' '):
        #word=word.replace('-',' ')
        #print(text2int(word))
        tmp_sentence,flag=text2int(word)
        sentence1+=str(tmp_sentence)+" "
        if(flag):
            word=word.split(' ')
            if(len(word)>1):
                sentence1+=pre_process_sentence("".join([str(elem) for elem in word[1:]]))

    sentence=sentence1
    sentence=sentence.replace(',', '')
    sentence=re.sub("[A-Za-z\-:()]+", lambda ele: " " + ele[0] + " ", sentence)
    return sentence
def evaluate_dataset():
  correct_ans=0
  total_ans=0
  idx=0
  normal_correct=0
  total_correct=0
  except_cnt=0
  for q in range(len(normal_questions_eval)):
    
    idx+=1
    #print(idx)
    if idx % 10 == 0 :
      #print("sadsadsad")
      print(except_cnt)
      print(correct_ans/total_ans)
      print(normal_correct/total_correct)
    normal_sentence=pre_process_sentence(normal_questions_eval[q])
    normal_sentence_lst=normal_sentence.split(' ')
    
    # print(prepared_questions_eval[q])
    prepared_sentence_lst=prepared_questions_eval[q].split(' ')
    
    mapping_dict=dict()
    #finding_idx=0
    predicted_tmp=list()
    for elem in normal_sentence_lst:
      #print(elem)
      if len(elem)!=0:
        if elem[-1]=="%":
          #print(elem[:-1])
          predicted_tmp.append(elem[:-1])
          predicted_tmp.append("%")
        else:
          predicted_tmp.append(elem)
      else:
        predicted_tmp.append(elem)
    
    normal_sentence_lst=predicted_tmp
    #print(normal_sentence_lst)
    #print(prepared_sentence_lst)
    for k in range(len(normal_sentence_lst)):
      if normal_sentence_lst[k] != prepared_sentence_lst[k]:
        mapping_dict[prepared_sentence_lst[k]]=normal_sentence_lst[k]
    #print(mapping_dict)
    predicted_prefix=predicted_eqn(prepared_questions_eval[q])
    #print(predicted_prefix)
    
    #print(predicted_prefix)
    #print(prefix_equations_list_eval[q])
    for j in range(len(predicted_prefix)):

      if j<len(prefix_equations_list_eval[q]) and predicted_prefix[j]==prefix_equations_list_eval[q][j]:
        normal_correct+=1
    total_correct+=len(predicted_prefix)
    
    predicted_prefix_temp=list()
    for elem in predicted_prefix:
      if (elem in mapping_dict.keys()):
        predicted_prefix_temp.append(mapping_dict[elem])
      else:
        predicted_prefix_temp.append(elem)
    predicted_prefix=predicted_prefix_temp
    
    #print(predicted_prefix)
   
    # for elem in predicted_prefix:
    #   print(elem)
    #   if len(elem)!=0:
    #     if elem[-1]=="%":
    #       predicted_tmp.append(elem[:-1])
    #     else:
    #       predicted_tmp.append(elem)
    #   else:
    #     predicted_tmp.append(elem)

    #predicted_prefix=predicted_tmp
    #print(predicted_prefix)
    
      
    
    try:

      predicted_ans=evaluate_prefix(predicted_prefix)
      if abs(predicted_ans-float(answers[q]))<0.005:
        correct_ans+=1
      total_ans+=1
    except:
      except_cnt+=1
      total_ans+=1
    print("Predicted Answer=",predicted_ans,"Expected Answer=",answers[q])
    
  return float(correct_ans/total_ans)
print("asasdsa")
print(evaluate_dataset())

asasdsa
Predicted Answer= -11.0 Expected Answer= 20
Predicted Answer= 72686.0 Expected Answer= 1466
Predicted Answer= 72686.0 Expected Answer= 180
Predicted Answer= 32.0 Expected Answer= 32
Predicted Answer= 39.76 Expected Answer= 39.76
Predicted Answer= 3.999999999999999 Expected Answer= 4
Predicted Answer= -0.1 Expected Answer= 10%
Predicted Answer= 42.0 Expected Answer= 105
Predicted Answer= 1764.0 Expected Answer= 2304
2
0.3333333333333333
0.5961538461538461
Predicted Answer= 19.625 Expected Answer= 12.5
Predicted Answer= 47.770334928229666 Expected Answer= 39
Predicted Answer= 0.10491071428571429 Expected Answer= 40
Predicted Answer= 10842480.0 Expected Answer= 158
Predicted Answer= 11.0 Expected Answer= 11
Predicted Answer= 85.0 Expected Answer= 85
Predicted Answer= 4500002.0 Expected Answer= 8000
Predicted Answer= 4500002.0 Expected Answer= 65
Predicted Answer= 39.285714285714285 Expected Answer= 6.93
Predicted Answer= 43.0 Expected Answer= 43
3
0.3157894736842105
0.550458715596

KeyboardInterrupt: ignored

### Get Accuracy and Corpus BLEU

#### Translation